In [8]:
using Convex, SCS

x0 = [-1.75,-1.75]
f = x -> (1-x[1])^2 + 5*(x[2] - x[1]^2)^2
∇f = x -> [2*(10*x[1]^3-10*x[1]*x[2]+x[1]-1), 10*(x[2]-x[1]^2)]
H = x -> [-20*(x[2] - x[1]^2) + 40*x[1]^2+2    -20*x[1]; 
          -20*x[1]                                  10] 

function trust_region_descent(f, ∇f, H, x, k_max;
    η1=0.25, η2=0.5, γ1=0.5, γ2=2.0, δ=2.0)
    y = f(x)
    
    for k in 1 : k_max
        x′, y′ = solve_trust_region_subproblem(f, ∇f, H, x, δ)
        r = (y - f(x′)) / (y - y′)
        if r < η1
            δ *= γ1
        else
            x, y = x′, y′
            if r > η2
                δ *= γ2
            end
        end
    end
    return x
end

function solve_trust_region_subproblem(f, ∇f, H, x0, δ)
    x = Variable(length(x0))
    p = minimize(f(x0) + ∇f(x0)'*(x-x0) + quadform(x-x0, H(x0))/2)
    p.constraints += norm(x-x0) <= δ
    solve!(p, SCS.Optimizer(verbose=false))
    return (x.value, p.optval)
end

solve_trust_region_subproblem (generic function with 1 method)

In [7]:
trust_region_descent(f, ∇f, H, x0, 200)

2×1 Array{Float64,2}:
 1.0000000000753706
 1.000000000109109